In [ ]:
import numpy as np
import qiskit
import os
import sys
import copy

from qiskit import Aer, QuantumCircuit, QuantumRegister, ClassicalRegister

import sympy as sym
import jax.numpy as jnp
from jax import grad, jit, vmap
from functools import partial, reduce
from typing import Optional, Callable, Union, List, Dict
from collections.abc import Iterable
from qiskit.aqua import QuantumInstance


from qiskit.aqua.operators import (I,X,Y,Z, ListOp, PauliOp, Zero, DictStateFn, SummedOp, ComposedOp, TensoredOp,
                                  OperatorBase, CircuitOp, CircuitStateFn, CircuitSampler,
                                  StateFn, PrimitiveOp, PauliExpectation,  One, Zero, OperatorStateFn)
from qiskit.circuit import ParameterVector, Parameter, ParameterExpression
from qiskit.circuit.library import RealAmplitudes, EfficientSU2

from qiskit.aqua.operators.gradients import GradientBase, Gradient, Hessian, QFI 
from qiskit.aqua.operators.gradients.gradient import GradientLinComb, GradientParamShift


# Defining an expectation value in Opflow
For more details regarding Opflow, see [this notebook made by it's creator](https://github.com/dongreenberg/aqua_talks/blob/master/Understanding%20Aqua's%20Operator%20Flow.ipynb)

In [18]:
#Define an Ansatz circuit
num_qubits = 3
circ = RealAmplitudes(num_qubits, reps=1)

#Wrap the Ansatz into a circuit operator object
circ_op = CircuitOp(circ)

#Print the Ansatz parameters
print('Ansatz parameters ', circ.ordered_parameters)

Ansatz parameters  [Parameter(θ[0]), Parameter(θ[1]), Parameter(θ[2]), Parameter(θ[3]), Parameter(θ[4]), Parameter(θ[5])]


In [19]:


print(circ_op)

     ┌──────────┐          ┌──────────┐            
q_0: ┤ RY(θ[0]) ├──■────■──┤ RY(θ[3]) ├────────────
     ├──────────┤┌─┴─┐  │  └──────────┘┌──────────┐
q_1: ┤ RY(θ[1]) ├┤ X ├──┼───────■──────┤ RY(θ[4]) ├
     ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    ├──────────┤
q_2: ┤ RY(θ[2]) ├─────┤ X ├───┤ X ├────┤ RY(θ[5]) ├
     └──────────┘     └───┘   └───┘    └──────────┘


In [20]:
#Define the Hamiltonian
H = 3*(I^Z^Z) + (4*(Z^I^X))

#Convert the Hamiltonian into an observable
H_op = ~StateFn(H)

#Combine the operator objects into the cost function object
E_op = H_op @ circ_op @ Zero
print('Cost function operator ', E_op)

Cost function operator  ComposedOp([
  OperatorMeasurement(SummedOp([
    3.0 * IZZ,
    4.0 * ZIX
  ])),
  CircuitStateFn(
       ┌──────────┐          ┌──────────┐            
  q_0: ┤ RY(θ[0]) ├──■────■──┤ RY(θ[3]) ├────────────
       ├──────────┤┌─┴─┐  │  └──────────┘┌──────────┐
  q_1: ┤ RY(θ[1]) ├┤ X ├──┼───────■──────┤ RY(θ[4]) ├
       ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    ├──────────┤
  q_2: ┤ RY(θ[2]) ├─────┤ X ├───┤ X ├────┤ RY(θ[5]) ├
       └──────────┘     └───┘   └───┘    └──────────┘
  )
])


In [36]:
#Initialize random parameters for the Ansatz
param_values = dict(zip(circ.ordered_parameters, 
                        np.random.rand(len(circ.ordered_parameters))))

#Convert into a gradient operator using either the parameter shift...
E_grad_op = Gradient(method='param_shift').convert(E_op, circ.ordered_parameters)

#Evaluate the cost function's gradients
E_grad = E_grad_op.assign_parameters(param_values).eval()
print('Parameter shift', np.around(E_grad, 3))

Parameter shift [ 0.132+0.j -2.321-0.j -0.908+0.j  2.431+0.j -2.08 -0.j -1.042+0.j]


In [39]:
#... or the linear combination method, ...
E_grad_op = Gradient(method='lin_comb').convert(E_op, circ.ordered_parameters)

#Evaluate the cost function's gradients
E_grad = E_grad_op.assign_parameters(param_values).eval()
print('Linear combination', np.around(E_grad, 3))

Linear combination [ 0.132+0.j -2.321-0.j -0.908+0.j  2.431+0.j -2.08 -0.j -1.042+0.j]


In [40]:
#... or a numerical finite-difference method.
E_grad_op = Gradient(method='fin_diff').convert(E_op, circ.ordered_parameters)

#Evaluate the cost function's gradients
E_grad = E_grad_op.assign_parameters(param_values).eval()
print('Finite difference ', np.around(E_grad, 3))

Finite difference  [ 0.126+0.j -2.225-0.j -0.871+0.j  2.331+0.j -1.994-0.j -0.999+0.j]


In [24]:

from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import CG

qi_sv = QuantumInstance(Aer.get_backend('statevector_simulator'),
                                             shots=1,
                                             seed_simulator=2,
                                             seed_transpiler=2)

wavefunction = circ
#Conjugate Gradient algorithm
optimizer = CG(maxiter=50)

E_grad_fn = Gradient().gradient_wrapper(E_op, circ.ordered_parameters, method='lin_comb', backend = qi_sv)

# Gradient callable
vqe = VQE(H, wavefunction, optimizer=optimizer, gradient_fn = E_grad_fn)
# vqe = VQE(H, wavefunction, optimizer=optimizer)
result = vqe.run(qi_sv)
print(result)

{'optimal_parameters': {Parameter(θ[0]): -1.2138300918228029, Parameter(θ[1]): -1.2690875327484805, Parameter(θ[2]): 2.0482032986552436, Parameter(θ[3]): 1.3430215798371643, Parameter(θ[4]): 2.482642991231282, Parameter(θ[5]): -0.6267399205912261}, 'optimal_point': [-1.2138300918228029, -1.2690875327484805, 2.0482032986552436, 1.3430215798371643, 2.482642991231282, -0.6267399205912261], 'optimal_value': -0.6801716312180073, 'optimizer_evals': 1, 'optimizer_time': 0.013243913650512695, 'eigenvalue': (-0.6801716312180073+0j), 'eigenstate': array([2.06782192e-01+0.j, 7.21008762e-01+0.j, 1.27816408e-01+0.j,
       3.55278061e-01+0.j, 5.73767133e-04+0.j, 3.94930831e-01+0.j,
       3.05473596e-01+0.j, 2.13408402e-01+0.j]), 'cost_function_evals': 1}


In [25]:
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import CG

#Backend
qi_sv = QuantumInstance(Aer.get_backend('statevector_simulator'), shots=1, seed_simulator=2, seed_transpiler=2)



In [42]:
#Conjugate Gradient
optimizer = CG(maxiter=50)

#Gradient
grad = Gradient(method='param_shift')

#VQE
vqe_grad = VQE(circ, optimizer=optimizer, gradient=grad, quantum_instance=qi_sv)

In [50]:
#Results
result_grad = vqe_grad.compute_minimum_eigenvalue(H)
print('Evaluated parameters:', result_grad['optimal_parameters'])
print('\nEvaluated ground state energy:', result_grad['optimal_value'])

Evaluated parameters: {Parameter(θ[0]): 2.548092176237614, Parameter(θ[1]): 0.7781613330485069, Parameter(θ[2]): -1.5707944090106967, Parameter(θ[3]): 0.6173396989642901, Parameter(θ[4]): -3.3956030681023326, Parameter(θ[5]): 1.5707954702259457}

Evaluated ground state energy: -4.999999999995852


In [28]:
# {'optimal_parameters': {Parameter(θ[0]): 4.161104709858437, Parameter(θ[1]): -3.9794435770040395, Parameter(θ[2]): -7.853980686234145, Parameter(θ[3]): -6.744118187995095, Parameter(θ[4]): 6.2327671120955594, Parameter(θ[5]): 4.712388685956566}, 'optimal_point': [4.161104709858437, -3.9794435770040395, -7.853980686234145, -6.744118187995095, 6.2327671120955594, 4.712388685956566], 'optimal_value': -4.9999999999976925, 'optimizer_evals': 42, 'optimizer_time': 14.205756902694702, 'eigenvalue': (-4.9999999999976925+0j), 'eigenstate': array([-4.38620963e-08+0.j, -5.01242223e-07+0.j,  2.44228445e-07+0.j,
#        -1.63577740e-07+0.j, -3.62290401e-01+0.j, -7.24580197e-01+0.j,
#         5.24388429e-01+0.j,  2.62194544e-01+0.j]), 'cost_function_evals': 42}

NameError: name 'θ' is not defined

In [ ]:
#Define a circuit that prepares our state
circ = RealAmplitudes(3, reps=1)
#Wrap the Ansatz into a circuit operator object
circ_op = CircuitOp(circ)


p = Parameter('p0')

#Define the observables we wish to measure
meas_bases = 3*(I^Z^Z) + (4*(Z^I^X))
meas_op = ~StateFn(meas_bases)

#Compose these opflow objects
expec_op = meas_op @ circ_op @ Zero
print(expec_op)

# Massage this operator into a form that we know how to evaluate

In [ ]:
converted_op = PauliExpectation().convert(expec_op)
print(converted_op)

params = circ.ordered_parameters
param_bindings = {param: 1 for param in params}
#print(param_bindings)


bound_op = expec_op.bind_parameters(param_bindings)
#print(bound_op)
#print("\nexpectation value = ",np.real(bound_op.eval()))

# Taking the gradient of this expectation value

In [ ]:
grad_op = Gradient().convert(expec_op, params, method='param_shift')
print(grad_op[0])

bound_grad_op = grad_op.bind_parameters(param_bindings)
#print(bound_grad_op)
gradients = np.real(bound_grad_op.eval())
print("Gradients = \n", gradients)

# How to extend this to Natural Gradients

Note: In practice we will need to do some more sophisticated processing in order to ensure this matrix inversion is well-behaved.

In [ ]:
#Get the operator which will evaluate to the Quantum Fisher Information Metrix tensor
# or an approximation of it
qfi_op = QFI().convert(circ_op@Zero, params, approx=None)
#print(qfi_op)
qfi = np.real(qfi_op.bind_parameters(param_bindings).eval())
print(qfi)

In [ ]:
# Multiply the gradients by the inverse of the QFI
qfi_inv = np.linalg.inv(qfi)
nat_grad = np.matmul(qfi_inv, gradients)
print(nat_grad)

In [ ]:
# Multiply the gradients by the inverse of the QFI
qfi_inv = np.linalg.inv(qfi)
nat_grad = np.matmul(qfi_inv, gradients)
print(nat_grad)

# Everything below here is scratch work

In [ ]:
a = Parameter('a')
b = Parameter('b')
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

coeff_0 = Parameter('c_0')
coeff_1 = Parameter('c_1')
H = coeff_0*coeff_0 * X + coeff_1*coeff_0 * Z
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
gradient_coeffs = [(coeff_0, coeff_0), (coeff_0, coeff_1), (coeff_1, coeff_1)]
coeff_grad = Hessian().convert(op, gradient_coeffs)
values_dict = [{coeff_0: 0.5, coeff_1: -1, a: np.pi / 4, b: np.pi},
               {coeff_0: 0.5, coeff_1: -1, a: np.pi / 4, b: np.pi / 4}]

correct_values = [[2*np.sqrt(2), 0, 0], [2 * np.sqrt(2), 2, 0]]

for i, value_dict in enumerate(values_dict):
    np.testing.assert_array_almost_equal(coeff_grad.assign_parameters(value_dict).eval(), correct_values[i],
                                         decimal=4)

In [ ]:
H = 0.5 * X - 1 * Z
a = Parameter('a')
b = Parameter('b')
params = [(a, a), (a, b), (b, b)]

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
state_hess = Hessian().convert(operator=op, params=params, method='param_shift')
# state_hess = HessianLinComb().convert(operator=op, params=params)
values_dict = [{a: np.pi / 4, b: np.pi}, {a: np.pi / 4, b: np.pi / 4},
               {a: np.pi / 2, b: np.pi / 4}]
correct_values = [[-0.5 / np.sqrt(2), 1 / np.sqrt(2), 0],
                  [-0.5 / np.sqrt(2) + 0.5, -1 / 2., 0.5],
                  [1 / np.sqrt(2), 0, 1 / np.sqrt(2)]]

for i, value_dict in enumerate(values_dict):
    np.testing.assert_array_almost_equal(state_hess.assign_parameters(value_dict).eval(), correct_values[i])

In [ ]:
-0.5 / np.sqrt(2)

In [ ]:
a = Parameter('a')
b = Parameter('b')
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

c0 = Parameter('c_0')
c1 = Parameter('c_1')
H = c0*c0 * X + c1*c0 * Z
coeffs = [(c0, c0), (c0, c1), (c1, c1)]
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
g = Hessian().convert(op, coeffs)
print('original_op\n----------\n', op)
print('\ngrad_op\n----------\n', g)

# You can see that the gradient code works here. 

In [ ]:
value_dict = {c0: 0.5, c1: -1, a: np.pi / 4, b: np.pi/4}
g_bound = g.assign_parameters(value_dict)

In [ ]:
print(g_bound)
g_bound.eval()

In [ ]:
cs = CircuitSampler(backend=Aer.get_backend('statevector_simulator'))

In [ ]:
sampled_g = cs.convert(g_bound)

In [ ]:
print(sampled_g)

In [ ]:
print(coeff_grad)

In [ ]:
print(g.reduce())

In [ ]:
print(op)
print('----')
print(d_g)
print('----')
print(dd_g)

In [ ]:
value_dict = {coeff_0: 0.5, coeff_1: -1, a: np.pi / 4, b: np.pi / 4}
correct_values = [2 * np.sqrt(2), 2, 0]


np.testing.assert_array_almost_equal(g.assign_parameters(value_dict).eval(), correct_values,
                                     decimal=4)

In [ ]:
value_dict = values_dict[0]
correct_values[0]

In [ ]:
co = PauliExpectation().convert(op)
print(co[0])

In [ ]:
sg = Gradient().convert(operator=co[0], params=params[0], method=method)


In [ ]:
print(sg)

In [ ]:
op0 = (sg[0].bind_parameters(value_dict).eval())
op1 = (sg[1].bind_parameters(value_dict).eval())

In [ ]:
lo = ListOp(oplist=[op0, op1], combo_fn = sg.combo_fn)

In [ ]:
sg.combo_fn([op0, op1])

In [ ]:
print(co[0])

In [ ]:
break

In [ ]:
go = sg.bind_parameters(value_dict)

print(go.eval())

In [ ]:
#Define a circuit that prepares our state
circ = RealAmplitudes(3, reps=1)
circ_op = CircuitOp(circ)
#print(circ_op)
params = circ.ordered_parameters

p = Parameter('p0')

#Define the observables we wish to measure
meas_bases = 2*p*(I^Z^Z) + (p*p + 5*p)*(Z^I^Z)
meas_op = ~StateFn(meas_bases)

#Compose these opflow objects
expec_op = meas_op @ circ_op @ Zero
prob_op = circ_op @ Zero

#grad_op = expec_op.autograd(p, 'param_shift')
grad_op = expec_op.autograd((params[0],params[1]), 'param_shift')

#grad_op = expec_op.autograd(p, 'param_shift')
#old_grad = Gradient().convert(expec_op, params[0], 'param_shift')

In [ ]:
print(expec_op)

In [ ]:
print(expec_op.autograd(params[0], 'fin_diff').reduce())

In [ ]:
print(prob_op.autograd(params[0], 'fin_diff'))

# LinComb Sketch

In [ ]:
eo = PauliExpectation(group_paulis=False).convert(expec_op)
print(eo)

In [ ]:
grad_op = GradientLinComb().convert(eo, params[0])

In [ ]:
grad_op = GradientLinComb().convert(eo, params[0])

In [ ]:
grad_op = GradientLinComb().convert(expec_op, params[0])

In [ ]:
print(grad_op)

In [ ]:
grad_op = prob_op.autograd(params[0], 'lin_comb')
print(grad_op)

In [ ]:
print(grad_op)
#print(old_grad)